In [1]:
from import_library import *
from model_stack import model_list

In [2]:
data_type ='mean' #mat, csv, mix
# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = '/Volumes/DatasetCGRD/DataSet/dFC matrix matlab/' #raw_data_path = './data/image data/dFC matrix matlab/'
# next_path ='mdALFF.csv' #next_path ='C0005FC_z.mat'
class loaders:
    def __init__(self, data_type='mat' ,mat_dtype='FCM_mean'):
        self.data_type = data_type
        if mat_dtype=='mean':
            md ='FCM_mean'
        else:
            md = 'FCM_var'
        self.mat_dtype = md
        self.concat_mat_array = []
    def mat_process(self, data):
        get_data = []
        for idx, i in enumerate(data):
            if len(i[idx+1::])!=0:
                get_data.append(list(i[idx+1::]))
        get_data = list(itertools.chain(*get_data))
        # print([idx for idx, i in enumerate(get_data) if (i>-2)==False])
        return get_data
    def load_mat(self, data_path=None):
        mat = scipy.io.loadmat(data_path)
        get_value = mat[self.mat_dtype ]
        get_value = self.mat_process(get_value)
        self.concat_mat_array.append(get_value)
    def callback(self,):
        return np.array(self.concat_mat_array).astype(np.float16)

In [3]:
from itertools import compress
# feature selection process
rank_threshold = 100
def feature_filter(patient_value, load_path = None,outcome_name = None, data_type = None):
    feature_rank_df = pd.read_csv('{}/{}_{}_{}.csv'.format(load_path, outcome_name,data_type, 'fc'), index_col=False)
    filter_feature_list = [feature_rank_df['rank'].values<=rank_threshold][0]
    new_patient_value = []
    new_feature = list(compress(feature_rank_df['feature_location'].values, filter_feature_list))
    for single_patient in patient_value:
        new_patient_value.append(list(compress(single_patient, filter_feature_list)))
    return np.array(new_patient_value), new_feature

In [4]:
grid =False
# result_dict={'outcome':[], 'model':[], 'Reg:mae(train)':[], 'Reg:mae(valid)': [], 'Reg:mse(train)':[], 'Reg:mse(valid)': [], 'Reg:mape(train)':[], 'Reg:mape(valid)': [], }
feature_selection_df_savepath = './feature_selection_result'
grid_stack = []
result_dict={'outcome':[], 'model':[], 'Reg:mae(train)':[], 'Reg:mae(valid)': [], 'Reg:mse(train)':[], 'Reg:mse(valid)': [], 'Reg:mape(train)':[], 'Reg:mape(valid)': [], }

def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        kfold = KFold(n_splits=4)
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    result_dict['outcome'].append(outcome_name)
                    result_dict['model'].append(model)
                    label_ = clinical_data[outcome_name]
                    print("FC {} Outcome: {}".format(data_type,outcome_name))
                    print("--Start Training {} model--".format(model))
                    if 'MOCA' in outcome_name:
                        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
                    elif 'AD8' in outcome_name:
                        label_s = [1 if i>=2 else 0 for i in list(label_)]
                        label_ = label_+1
                    ld = loaders(data_type = 'mat', mat_dtype = data_type)
                    for next_path in sorted(os.listdir(raw_data_path)):
                        if 'C0018FC_z.mat' not in next_path:
                            
                            ld.load_mat(data_path = os.path.join(raw_data_path,next_path))
                    patient_value_index = clinical_data['subj']
                    patient_value = ld.callback()
                    patient_value, _ = feature_filter(patient_value, load_path = feature_selection_df_savepath, outcome_name = outcome_name, data_type = data_type, )
                    print(patient_value.shape)
                    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
                    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
                    X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
                    y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)
                    print("Feature Number: {}".format(X_train.shape[1]))
                    if True:
                        if grid==False:
                            print("FC Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                            print("--aStart Training {} model--".format(model))
                            if model in 'MLPR':
                                model = model_stack[model].fit(X_train, y_train)
                            else:
                                
                                model = model_stack[model].fit(np.array(X_train), np.array(y_train), eval_set=[[np.array(X_valid), np.array(y_valid)]], eval_metric="mape", verbose=False)
                            train_pred =  model.predict(X_train)
                            
                            valid_pred =  model.predict(X_valid)
                        else:
                            print("FC Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                            print("--Start Training {} model--".format(model))
                            GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                    return_train_score=True)
                            GridModel.fit(np.array(X_train).astype(np.float16), np.array(y_train).astype(np.int8))
                            param_grid = model_stack[model][1]
                            print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                            train_pred =  GridModel.predict(X_train)
                            valid_pred =  GridModel.predict(X_valid)
                            print("Data Length. Training : {} || Validation: {}".format(len(y_train), len(y_valid)))
                            grid_stack.append(GridModel)
                        result_dict['Reg:mae(train)'].append(round(metrics.mean_absolute_error(y_train,train_pred), 5))
                        result_dict['Reg:mae(valid)'].append(round(metrics.mean_absolute_error(y_valid,valid_pred), 5))
                        result_dict['Reg:mse(train)'].append(round(metrics.mean_squared_error(y_train,train_pred),5))
                        result_dict['Reg:mse(valid)'].append(round(metrics.mean_squared_error(y_valid,valid_pred),5))
                        result_dict['Reg:mape(train)'].append(round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                        result_dict['Reg:mape(valid)'].append(round(metrics.mean_absolute_percentage_error(y_valid,valid_pred),5))
                        
                        # print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                        # print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,test_pred),5))
                        # print('Test MAE: ', round(metrics.mean_absolute_error(y_test,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_test,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_test,test_pred),5))
                        print('----------------------------------------------------')
                # print('\n')
                

FC mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training MLPR model--
(84, 99)
Feature Number: 99
FC Outcome: AD8主觀認知障礙[0-1,>=2][mean] - cut rank point: 100 - GridSearch: False
--aStart Training MLPR model--
----------------------------------------------------
FC mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training XBGR model--
(84, 99)
Feature Number: 99
FC Outcome: AD8主觀認知障礙[0-1,>=2][mean] - cut rank point: 100 - GridSearch: False
--aStart Training XBGR model--
----------------------------------------------------
FC mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training LGBMR model--
(84, 99)
Feature Number: 99
FC Outcome: AD8主觀認知障礙[0-1,>=2][mean] - cut rank point: 100 - GridSearch: False
--aStart Training LGBMR model--
----------------------------------------------------
FC mean Outcome: MOCA客觀認知測驗分數[<=23,>=24]
--Start Training MLPR model--
(84, 101)
Feature Number: 101
FC Outcome: MOCA客觀認知測驗分數[<=23,>=24][mean] - cut rank point: 100 - GridSearch: False
--aStart Training MLPR model--
-----------

In [5]:
for i in grid_stack:
    print(i.best_params_)
    print('---------------------------------------------------------------------------------------------------------------')

In [6]:
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [FC - {} - Grid: {}]".format(data_type, grid))

,outcome,model,Reg:mae(train),Reg:mae(valid),Reg:mse(train),Reg:mse(valid),Reg:mape(train),Reg:mape(valid)
0,"AD8主觀認知障礙[0-1,>=2]",MLPR,1.184520,1.643980,6.846920,6.540790,0.248300,0.546370
1,"AD8主觀認知障礙[0-1,>=2]",XBGR,0.000210,1.357900,0.000000,4.653610,0.000120,0.798820
2,"AD8主觀認知障礙[0-1,>=2]",LGBMR,1.858480,1.835690,5.273670,4.648710,1.037240,1.020630
3,"MOCA客觀認知測驗分數[<=23,>=24]",MLPR,3.985860,5.302440,41.926020,40.042040,0.159830,0.207640
4,"MOCA客觀認知測驗分數[<=23,>=24]",XBGR,0.000280,1.874800,0.000000,5.183270,0.000010,0.078670
5,"MOCA客觀認知測驗分數[<=23,>=24]",LGBMR,3.078040,2.750000,12.888660,10.476260,0.126990,0.113440
